<a href="https://colab.research.google.com/github/ExTimeGameing/pythonMIREA/blob/main/fivePrakt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Задача1
import doctest


def add(a, b):
    """
    Даны два числа а и б, вернуть их сумму.
    :param a: int
    :param b: int
    :return: int

    >>> add(2, 3)
    5

    >>> add(3, 3)
    10

    >>> add(2, "С")
    """
    try:
        a = int(a)
        b = int(b)
        return a + b
    except:
        raise NameError("НЕ ЧИСЛО")

import doctest
doctest.testfile("testCode")


In [ ]:
#Задача3
# pip install pytest
# pip install coverage
# coverage run -m pytest main.py
# coverage report -m

def incr(x):
    if True == True:
        x += 1
    else:
        print("Пасхалка")
    return x


def test_incr():
    assert incr(0) == 1


In [ ]:
# Задача №4
import random
from collections import defaultdict
import inspect
from time import time
from ast import *


class Locator(NodeVisitor):
    def __init__(self):
        self.locs = defaultdict(list)

    def visit(self, node):
        self.locs[type(node)].append(node)
        self.generic_visit(node)


class Mutator(NodeTransformer):
    bin_operators = {
        Add: [Sub, Mult, Div, FloorDiv, Mod, LShift, RShift, BitOr, BitAnd, Pow],
        Sub: [Add, Mult, Div, FloorDiv, Mod, LShift, RShift, BitOr, BitAnd, Pow],
        Mult: [Sub, Add, Div, FloorDiv, Mod, LShift, RShift, BitOr, BitAnd, Pow],
        Div: [Sub, Mult, Add, FloorDiv, Mod, LShift, RShift, BitOr, BitAnd, Pow],
        FloorDiv: [Sub, Mult, Div, Add, Mod, LShift, RShift, BitOr, BitAnd, Pow],
        Mod: [Sub, Mult, Div, FloorDiv, Add, LShift, RShift, BitOr, BitAnd, Pow],
        LShift: [Sub, Mult, Div, FloorDiv, Mod, Add, RShift, BitOr, BitAnd, Pow],
        RShift: [Sub, Mult, Div, FloorDiv, Mod, LShift, Add, BitOr, BitAnd, Pow],
        BitOr: [Sub, Mult, Div, FloorDiv, Mod, LShift, RShift, Add, BitAnd, Pow],
        BitAnd: [Sub, Mult, Div, FloorDiv, Mod, LShift, RShift, BitOr, Add, Pow],
        Pow: [Sub, Mult, Div, FloorDiv, Mod, LShift, RShift, BitOr, BitAnd, Add]
    }

    bool_operators = {
        And: Or,
        Or: And
    }

    compare_operators = {
        Eq: [NotEq, Lt, LtE, Gt, GtE],
        NotEq: [Eq, Lt, LtE, Gt, GtE],
        Lt: [NotEq, Eq, LtE, Gt, GtE],
        LtE: [NotEq, Lt, Eq, Gt, GtE],
        Gt: [NotEq, Lt, LtE, Eq, GtE],
        GtE: [NotEq, Lt, LtE, Gt, Eq],
        Is: [NotEq, Lt, LtE, Gt, GtE, Eq, IsNot],
        IsNot: [NotEq, Lt, LtE, Gt, GtE, Is, Eq],
        In: [NotEq, Lt, LtE, Gt, GtE, Eq, NotIn],
        NotIn: [NotEq, Lt, LtE, Gt, GtE, In, Eq]
    }

    def visit_BinOp(self, node):
        if type(node.op) in self.bin_operators.keys():
            new_op = random.choice(self.bin_operators[type(node.op)])
            new_node = BinOp(
                left=node.left,
                op=new_op(),
                right=node.right
            )
            copy_location(new_node, node)
            fix_missing_locations(new_node)
            return new_node
        return node

    def visit_BoolOp(self, node):
        if type(node.op) in self.bool_operators.keys():
            new_op = random.choice(self.bool_operators[type(node.op)])
            new_node = BoolOp(
                values=node.values,
                op=new_op()
            )
            copy_location(new_node, node)
            fix_missing_locations(new_node)
            return new_node
        return node

    def visit_Compare(self, node):
        op_ind = random.choice(range(len(node.ops)))
        op = node.ops[op_ind]
        if type(op) in self.compare_operators.keys():
            new_ops = node.ops
            new_op = random.choice(self.compare_operators[type(op)])
            new_ops[op_ind] = new_op()
            new_node = Compare(
                left=node.left,
                ops=new_ops,
                comparators=node.comparators
            )
            copy_location(new_node, node)
            fix_missing_locations(new_node)
            return new_node
        return node

    def visit_Constant(self, node):
        if isinstance(node.value, int):
            new_node = Constant(
                value=node.value + random.choice([-1, 1])
            )
            copy_location(new_node, node)
            fix_missing_locations(new_node)
            return new_node
        return node

    def __init__(self, locs):
        random.seed(time())
        mutant_types = {BinOp, BoolOp, Compare, Constant}
        node_type = None
        while node_type not in mutant_types:
            node_type = random.choice(list(locs.keys()))
        self.target = random.choice(locs[node_type])

    def visit(self, node):
        if self.target != node:
            return self.generic_visit(node)
        return super(Mutator, self).visit(node)


def mutate_code(src, max_changes):
    tree = parse(src)
    loc = Locator()
    loc.visit(tree)
    mut = Mutator(loc.locs)
    for _ in range(random.randint(1, max_changes)):
        mut.visit(tree)
    return unparse(tree)


def make_mutants(func, size, max_changes):
    mutant = src = unparse(parse(inspect.getsource(func)))
    mutants = [src]
    time_start = time()
    while len(mutants) < size + 1:
        time_end = time()
        while mutant in mutants:
            mutant = mutate_code(src, max_changes)
            time_end = time()
            if time_end - time_start > 2:
                break
        if time_end - time_start > 2:
            break
        mutants.append(mutant)
    return mutants[1:]


def mut_test(func, test, size=20, max_changes=1):
    survived = []
    mutants = make_mutants(func, size, max_changes)
    print('Мутанты:')
    print('Количество мутантов: {}'.format(len(mutants)))
    print(*mutants, sep='\n')
    for mutant in mutants:
        try:
            exec(mutant, globals())
            test()
            survived.append(mutant)
        except:
            pass
    print('\nВыжило:')
    print('Количество выживших: {}'.format(len(survived)))
    print(*survived, sep='\n')


def foo(n):
    if (n % 2 == 0):
        return n + 5
    else:
        return n - 5


def test():
    func = globals()['foo']
    assert func(10) == 15


if __name__ == '__main__':
    mut_test(foo, test, 100)


In [ ]:
# Задача №6
from hypothesis import given, strategies


# Пункт №1 Тестирование для чисел, строк, списков

def add(x, y):
    return x + y

def subtracr(x, y):
    return x - y

@given(x=strategies.integers(min_value=5), y=strategies.integers(min_value=1))
def test_add(x, y):
    # assert add(x, y) == add(x, y) * (-1)
    assert add(x, y) == add(y, x) # Пункт №3 - Свойство "Разные пути, одна цель"
    assert subtracr(add(x, y), y) == x #Пункт №3 - Свойство "Туда и обратно"

def check(s, str):
    if s in str:
        return True
    else:
        return False


@given(s=strategies.text(), str=strategies.text())
def test_check(s, str):
    assert check(s, str) == True


def mode(data):
    res = len(data)
    data.sort()
    res2 = len(data)
    return res == res2


@given(data=strategies.lists(strategies.integers(min_value=1, max_value=100), min_size=5))
def test_mode(data):
    print(data)
    assert mode(data) == True  # Пункт №3 - Свойство "Некоторые вещи не меняются"


# test_add()
# test_check()
test_mode()

# Пункт №2 Тестирование словарей, деревьев, графов
def dictCount(d):
    return len(d)


keys = strategies.integers()
vals = strategies.text()


@given(d=strategies.dictionaries(keys=keys, values=vals))
def test_dictCount(d):
    assert dictCount(d) < 5


def treeKidsCount(t):
    return len(t)


@given(t=strategies.lists(elements=strategies.lists(elements=strategies.booleans(), min_size=2, max_size=3)))
def test_tree(t):
    assert treeKidsCount(t) < 4


N = [[0, 1, 1, 1, 1, 1, 0, 0],  # a
     [0, 0, 1, 0, 1, 0, 0, 0],  # b
     [0, 0, 0, 1, 0, 0, 0, 0],  # c
     [0, 0, 0, 0, 1, 0, 0, 0],  # d
     [0, 0, 0, 0, 0, 1, 0, 0],  # e
     [0, 0, 1, 0, 0, 0, 1, 1],  # f
     [0, 0, 0, 0, 0, 1, 0, 1],  # g
     [0, 0, 0, 0, 0, 1, 1, 0]]  # h


def graph_weight(a, b):
    if 0 < a < 8 and 0 < b < 8:
        if N[a][b] == 1:
            return 1
        else:
            return 0


@given(x=strategies.integers(min_value=0, max_value=7), y=strategies.integers(min_value=0, max_value=7))
def test_graph(x, y):
    print(x, y)
    assert graph_weight(x, y) != 0


# test_dictCount()
# test_tree()
# test_graph()
